In [ ]:
!pip install snorkel
!pip install utils

In [ ]:
# import needed libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from snorkel.labeling import labeling_function


# Load the data and Preprocess it

if avalaible, small set of hand-labeled dataset will be useful as test set, needed for validating our classifier later

In [ ]:
# Load the dataset
# Load and Preprocess the data
filepath = 'https://raw.githubusercontent.com/CDU-data-science-team/pxtextmining/main/datasets/text_data.csv'
df = pd.read_csv(filepath, usecols=['feedback', 'label'])#, nrows=100)
print(f'{df.head()} \n\nno. of rows, columes: {df.shape} \n')

#  take a sample to easily test pipeline
# df = df.sample(100)

#fill missing value has this causes runtime error while fiting the model 
print(f'Missing values\n{df.isna().sum()}')
df['feedback'].fillna('Nothing', inplace = True)
print(df.isna().sum())

df_train = df

(9291, 2)


### Pipeline testing data

In [ ]:
# Define the label mappings for convenience
ABSTAIN = -1
NOT_SPAM = 0
SPAM = 1

#generate sample labels for testing current pipeline
import random

df_train['label'] =  [random.choice([SPAM, NOT_SPAM, ABSTAIN]) for _ in df_train.label]   # generate random label
df_train.rename(columns = {'feedback':'text'}, inplace = True)  # change the column name from feedback to text
df_train.head()

,label,text
0,0,Nothing.
1,0,Temperature in theatre a little low.
2,1,Same service available at Bingham Health Centre.
3,0,Appointment details given over phone - no phys...
4,0,On one occasion I was not made aware that my a...


# Labeling Functions
This are noisy, programmatic rules and heuristics that assign labels to unlabeled training data. They help users __encode domain knowledge__ and other supervision sources programmatically.


---
## Recommended practice for LF development
Typical LF development cycles include multiple iterations of ideation, refining, evaluation, and debugging. A typical cycle consists of the following steps:



1.   Look at examples to generate ideas for LFs
2.   Write an initial version of an LF
3. Spot check its performance by looking at its output on data points in the training set (or development set if available)
4. Refine and debug to improve coverage or accuracy as necessary

The goal for LF development is to create a high quality set of training labels for our unlabeled dataset, not to label everything or directly create a model for inference using the LFs. The training labels are used to train a separate discriminative model (in this case, one which just uses the comment text) in order to generalize to new, unseen data points. Using this model, we can make predictions for data points that our LFs don’t cover.


### Writing series of Labeling Functions

In [ ]:
from snorkel.labeling import labeling_function

# Below are examples of rules been converted into functions (speak to SME's to set the rules)

# Keyword matches:
@labeling_function()
def lf_keyword_my(x):
    """Many spam comments talk about 'my channel', 'my video', etc."""
    return SPAM if "my" in x.text.lower() else ABSTAIN

# Regular expressions related rule
import re
@labeling_function()
def lf_regex_check_out(x):
    """Spam comments say 'check out my video', 'check it out', etc."""
    return SPAM if re.search(r"check.*out", x.text, flags=re.I) else ABSTAIN

# Use pre-trained model (we use sentiment analysis here)
from textblob import TextBlob
@labeling_function()
def lf_textblob_polarity(x):
    """
    We use a third-party sentiment classification model, TextBlob.
    We combine this with the heuristic that non-spam comments are often positive.
    """
    return NOT_SPAM if TextBlob(x.text).sentiment.polarity > 0.3 else ABSTAIN

## Combine the LFs and apply LabelModel
This generate  the initial set of labels and Compare it to original label (if available)

we develop a <generative model> using LabelModel to combine the outputs of the LFs and generate noise-aware probabilistic (or confidence-weighted) label per data point

In [ ]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

# Define the set of labeling functions (LFs)
lfs = [lf_keyword_my, lf_regex_check_out, lf_textblob_polarity]
applier = PandasLFApplier(lfs)

# Apply the LFs to the unlabeled training data
L_train = applier.apply(df_train)

# Use LabelModel to automatically estimate the accuracies and correlations, reweight and combine the LF labels 
label_model = LabelModel(cardinality=2, verbose=True)   # cardinality is the number of classes
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)

# Generate noise-aware probabilistic set of clean, integrated labels 
predict_proba
probs_train = label_model.predict_proba(L=L_train, tie_break_policy="abstain")    #predict(L) - to Return label predictions (int).
# df_train["new_label"] = label_model.predict(L=L_train, tie_break_policy="abstain")  

# filter out unlabeled data points (this can hurt our model)
# (When all of the labeling functions abstain, the LabelModel abstains as well)
from snorkel.labeling import filter_unlabeled_dataframe
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)
# or
# df_train = df_train[df_train.new_label != ABSTAIN] 

100%|██████████| 9291/9291 [00:03<00:00, 2924.53it/s]


### Compute labeling accuracy

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, plot_confusion_matrix

preds_test = probs_to_preds(probs_train_filtered)
balanced_score = balanced_accuracy_score(df_train.label, preds_test)
acc_score = accuracy_score(df_train.label, preds_test) 
# or 
# label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")["accuracy"]

print(acc_score, balanced_score)


# balanced_score = balanced_accuracy_score(df_train.label, df_train.new_label)
# acc_score = accuracy_score(df_train.label, df_train.new_label) 
# or 

0.3315036056398665 0.3318184225490089


## Final model for Label generation and future Labeling (using Discriminative Model)

We train a machine learning model that can generalize beyond the coverage of the labeling functions and the LabelModel

For this, we can augment the existing labeled training datasets by creating transformed copies of the data points. **Data augmentation** is a practical and powerful method for injecting information about domain invariances into ML models via the data, rather than by trying to modify their internal architectures. e.g. randomly replacing a word with a synonym. We use the `transformation_function` (TF) for this purpose:

### Model Training
***Training Set***: The largest split of the dataset, and the one without any ground truth (“gold”) labels. We will generate labels for these data points with weak supervision.

***Test Set***: this is a small, standard held-out blind hand-labeled set for final evaluation of our discriminative classifier. This set should only be used for final evaluation, not error analysis. Depending on the approach the test set can be further splited into *development (validation) set* and *development split*. the **Development set** is used  for hyperparameter tuning of the end discriminative classifier

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# split the data into train and test set (for demostration purpose)
train_sample = 6000
train_text = df_train.text.tolist()[:train_sample]
test_text = df_train.text.tolist()[train_sample:]

# preprocess the data
#X_train = CountVectorizer(ngram_range=(1, 2)).fit_transform(train_text)
pre_pro = CountVectorizer(ngram_range=(1, 2)).fit(train_text)
X_train = pre_pro.transform(train_text)
X_test = pre_pro.transform(test_text)

clf = LogisticRegression(solver="lbfgs")
clf.fit(X=X_train, y=df_train.label.values[:train_sample])

y_true = df_train.label.values[train_sample:]
y_pred = clf.predict(X=X_test)
m_acc_score = accuracy_score(y_true, y_pred)
m_balanced_score = balanced_accuracy_score(y_true, y_pred)

print(m_acc_score, m_balanced_score)

0.3290793072014585 0.3319815389969851


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
